In [2]:
## De functie die tf-records leest en ze uiteindelijk weer wegschrijft (maar dan met minder labels) heet readTfRecord
## Deze functie roept andere functies aan: getLabels en getLabelIndices

import tensorflow as tf
import numpy as np
import pandas as pd
import re
import os
from collections import Counter
from tensorflow import gfile


/home/hugo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
# path
#train_data_pattern = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/bal_train/*.tfrecord'
train_data_pattern = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/bal_train/*.tfrecord'
train_data_folder_new = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/bal_train_new_labels'
test_data_pattern = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/eval/*.tfrecord'
test_data_folder_new = '/home/hugo/jupyter-workspace/data/audioset/audioset_v1_embeddings/eval_new_labels'

In [16]:
write_tfrecords_without_labels = False

In [9]:
## Keep list of all new labels (to see which ones occur often)
all_labels = []

In [10]:
## Open old csv file and alternated csv file with the super classes
mid_to_label_old = pd.read_csv('audioset/class_labels_indices.csv')
mid_to_label_old

,index,mid,display_name
0,0,/m/09x0r,Speech
1,1,/m/05zppz,"Male speech, man speaking"
2,2,/m/02zsn,"Female speech, woman speaking"
3,3,/m/0ytgt,"Child speech, kid speaking"
4,4,/m/01h8n0,Conversation
5,5,/m/02qldy,"Narration, monologue"
6,6,/m/0261r1,Babbling
7,7,/m/0brhx,Speech synthesizer
8,8,/m/07p6fty,Shout
9,9,/m/07q4ntr,Bellow


In [11]:
mid_to_label_new = pd.read_csv('audioset/class_labels_indices_new.csv',quotechar = '"')
mid_to_label_new

,index,mid,display_name,superclass
0,0,/m/09x0r,Speech,Speech
1,1,/m/05zppz,"Male speech, man speaking",Speech
2,2,/m/02zsn,"Female speech, woman speaking",Speech
3,3,/m/0ytgt,"Child speech, kid speaking",Speech
4,4,/m/01h8n0,Conversation,Speech
5,5,/m/02qldy,"Narration, monologue",Speech
6,6,/m/0261r1,Babbling,Speech
7,7,/m/0brhx,Speech synthesizer,Speech
8,8,/m/0jbk,Animal,Animal
9,9,/m/068hy,"Domestic animals, pets",Animal


In [12]:
#### Find new label for each of old label (527 -> 31)

def getNewLabel(label_old):
    ## Get m-id that matches with index
    mid = mid_to_label_old.loc[mid_to_label_old['index'] == label_old, 'mid']
    #print('mid : ',mid)

    ## Find the new label belonging to this m-id
    label_new = mid_to_label_new.loc[mid_to_label_new['mid'] == str(mid.values[0]), 'index']
    #print('label_new : ',label_new)
    
    return label_new

In [13]:
def createExample(example_old, new_labels):
    
    #print("New labels: " + str(new_labels))
    
    audio_embedding = example_old.feature_lists.feature_list['audio_embedding']
    feature_lists = tf.train.FeatureLists(feature_list={"audio_embedding": audio_embedding})

    label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=new_labels))
    video_id = example_old.context.feature['video_id'].bytes_list.value
    videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id)) 

    context_feats = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat})

    sequence_example = tf.train.SequenceExample(context=context_feats, feature_lists=feature_lists)
    
    return sequence_example

In [14]:
#### Lees tf-records en iterate over de examples#### Lee 
#### Voor elk example wordt de video-id gepakt en de bijbehorende lijst m-id's gezocht (in balanced_segment.csv)

## One tf-record has several examples
## This function checks for each example if it has a label we are interested in
## If so, we compute the one-hot vector
## Input: filename of tf-record
## Result: write new tf-record

def readWriteTfRecord(tfrecords_filename, new_folder):
    
    ## Create writer that writes new tf-record in train_data_folder_new
    path_tfrecord_new = new_folder + '/' + tfrecords_filename.split('/')[-1]
    writer = tf.python_io.TFRecordWriter(path_tfrecord_new)
    nr_new_examples = 0
    
    #print(path_tfrecord_new)
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)
    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels and find for each label the m-code in class_label_indices_old
        labels = example.context.feature['labels'].int64_list.value
        #print('labels : ', labels)
        
        new_labels = []
        for label in labels:
            ## hij returnt een set van cellen (of leeg of van 1 rij)
            label_new = getNewLabel(label)
            
            if(label_new.shape[0]>0):
                new_labels.append(label_new.values[0])
                #print('old : ',label,' new : ',label_new)
        ## If any of the labels left (for our model), write new tf-record
        if len(new_labels) > 0 or write_tfrecords_without_labels:

            ## Ad to all labels
            all_labels.extend(new_labels)
            
            sequence_example = createExample(example, new_labels)
            nr_new_examples += 1
            
            ## Write the example
            writer.write(sequence_example.SerializeToString())
            #print('Example gemaakt! Video-id: ' + str(example.context.feature['video_id'].bytes_list.value))

    #print("nr new examples: " + str(nr_new_examples))
    # Only write if any examples created

    if nr_new_examples > 0:
        writer.close()
        #print("tfrecord written")
    else:
        #print("empty file.. remove: " + path_tfrecord_new)
        os.remove(path_tfrecord_new)
        
    return(all_labels)

In [ ]:
# train_data_pattern = 'tfrecords/train_data_old/*.tfrecord'

## Read and rewrite all train files
files = gfile.Glob(train_data_pattern) 

for file in files:
    ## Read a file and rewrite it
    readWriteTfRecord(file, train_data_folder_new)
    

In [3]:
cnt_labels_train = Counter(all_labels)
print(cnt_labels_train.most_common(31))

NameError: name 'all_labels' is not defined

In [17]:
# test_data_pattern = 'tfrecords/test_data_old/*.tfrecord'

## Read and rewrite all test files
files = gfile.Glob(test_data_pattern)

for file in files:
    
    ## Read a file and rewrite it
    al = readWriteTfRecord(file, test_data_folder_new)

In [7]:
test_data_pattern

NameError: name 'test_data_pattern' is not defined

In [18]:
cnt_labels_train_and_test = Counter(all_labels)
print(cnt_labels_train_and_test.most_common(31))

[(12, 5695), (0, 5324), (14, 780), (8, 698), (13, 406), (9, 348), (15, 239), (3, 211), (26, 174), (17, 172), (10, 115), (18, 113), (19, 100), (16, 90), (25, 77), (2, 75), (1, 63), (11, 63), (23, 60), (20, 60), (29, 60), (30, 60), (4, 60), (27, 60), (6, 60), (28, 60), (21, 60), (22, 60), (24, 60), (5, 60), (7, 60)]


In [52]:
###########################################################################################
# Helper: show Examples in tfrecord file (only run this if you want to see just one example)
filename = train_data_folder_new + '/-1.tfrecord'
#filename = train_data_pattern.split('/*')[0] + '/-1.tfrecord'
record_iterator = tf.python_io.tf_record_iterator(path=filename)

# [137, 220, 519]
# [135, 137]
# [27, 466]
# [137, 264]

for string_record in record_iterator:

    ## Take one example
    example = tf.train.SequenceExample()
    example.ParseFromString(string_record)
    print(example.context.feature['labels'].int64_list.value)

[27]
